In [ ]:
import polars as pl
import pandas as pd
import polars.selectors as cs

In [ ]:
url_temp = "https://www.ncei.noaa.gov/pub/data/paleo/historical/phenology/japan/LatestVersion/759Temp7.xls"
url_flower = "https://www.ncei.noaa.gov/pub/data/paleo/historical/phenology/japan/LatestVersion/KyotoFullFlower7.xls"

temp_raw = pd.read_excel(url_temp, header=None)
flower_raw = pd.read_excel(url_flower, header=None)

In [ ]:
def map_null(x: pl.Series, value=-999.9):
    return x.map_dict({value: None}, default=pl.first())


# year, reconstructed temp, obs temp; -999.9 means no data
temp = (
    pl.from_pandas(temp_raw.loc[15:])
    .rename({"0": "year", "1": "temp_reconstructed", "2": "temp_observed"})
    .with_columns(cs.starts_with("temp_").pipe(map_null))
    .filter(
        pl.col("temp_reconstructed").is_not_null()
        | pl.col("temp_observed").is_not_null()
    )
)

temp

In [ ]:
def parse_date(year: pl.Series, monthday: pl.Series):
    """Given year like 821 and monthday like 411, produce 0821-04-11"""
    monthday_str = monthday.cast(str)
    month = pl.concat_str([pl.lit("0"), monthday_str.str.slice(0, 1)])
    day = monthday_str.str.slice(1, 2)
    date = pl.concat_str([year, month, day], separator="-").str.strptime(
        pl.Date, "%Y-%m-%d"
    )
    return date


def parse_day_of_year(year: pl.Series, doy: pl.Series):
    """
    Given year like 821 and day of year like 3, return 0821-01-03

    For this dataset, it gives the same result as using the monthday construct
    """
    jan1 = pl.format("{}-01-01", year).str.strptime(pl.Date, "%Y-%m-%d")
    days = pl.format("{}d", doy - 1)
    return jan1.dt.offset_by(days)


flower = (
    pl.from_pandas(flower_raw.loc[26:])
    .rename({"0": "year", "1": "day_of_year", "2": "date_in_year"})
    .with_columns(date=parse_date(pl.col("year"), pl.col("date_in_year")))
    .filter(pl.col("date").is_not_null())
    .select(["year", "date", "day_of_year"])
)

flower

In [ ]:
df = flower.join(temp, on="year", how="outer", validate="1:1")

df.write_csv("data/aono.csv")

df